<a href="https://colab.research.google.com/github/HsuLiSheng/big-data/blob/main/5_%E7%B6%B2%E7%AB%99%E7%9B%B8%E7%B0%BF%E4%B8%8B%E8%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###Colab要先安裝才能使用selenium
###!pip install selenium
###!apt-get update  # to update ubuntu to correctly run apt install
###!apt install chromium-chromedriver

In [1]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!apt-get update 

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 172 kB in 3s (54.4 kB/s)
Reading package lists... Don

In [3]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (107.0.5304.87-0ubuntu11.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.


In [4]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os,time

###沒設定參數的話，每次selenium開啟chrome瀏覽器會是個裸瀏覽器
chrome_options = webdriver.ChromeOptions()

###add_argument:添加啟動參數
###不顯示瀏覽器，在無桌面環境下啟動
chrome_options.add_argument('--headless')
###以root身分執行瀏覽器
chrome_options.add_argument('--no-sandbox')
###使用/tmp而非/dev/shm作為暫存區
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)

url = 'https://running.biji.co/index.php?q=album&act=photo_list&album_id=48414&cid=10128&type=album&subtitle=%E5%85%AB%E7%99%BE%E5%A3%AF%E5%A3%AB%E8%B6%85%E9%A6%AC%E7%B3%BB%E5%88%97-%E5%88%B6%E9%9C%B8%E7%92%B0%E5%8C%96100mile%E8%B6%85%E9%A6%AC-%E8%B5%B7%E8%B7%91%E9%BB%9E'

driver.get(url)

In [25]:
###隱性等待:最多等待幾秒鐘
driver.implicitly_wait(1)


###向下捲動10次
for i in range(1,11):
  ###將捲軸向下捲動
  driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  time.sleep(1)

soup = BeautifulSoup(driver.page_source, 'html.parser')
###取得標題，以select方式找到album-title類別中的第一筆資料，取得內容後濾除前後空白字元傳回變數
title = soup.select('.album-title')[0].text.strip() 
print(title)

###找到所有img標籤中類別名稱為photo_img photo-img的資料，回傳一個串列
all_image = soup.find_all('img', {"class":"photo_img photo-img"})

###建立目錄儲存照片
path = '/content/drive/MyDrive/Colab Notebooks/big data sample/photo'
if not os.path.exists(path):
  os.mkdir(path)

###紀錄下載了幾張圖片
n=0

for img in all_image:
  ###讀取照片網址的屬性
  src = img.get('src')
  ###判斷是否含有.jpg
  if src != None and ('.JPG' in src):
    full_path = src
    ###取得圖檔名(把前面共同網址去掉)
    filename = full_path.split('/')[-1] 

    ###圖檔可能會因為權限無法讀取，以try-except避免程式被中斷
    try:
      ###以urlopen讀取圖檔
      image = urlopen(full_path)
      ###open建立圖檔儲存的路徑和名稱
      with open(os.path.join(path,filename),'wb') as f:
        ###以write儲存圖檔
        f.write(image.read())
      n += 1
      ###最多下載100張照片
      if n>=100:
        break  
      elif n%10 == 0:
        print('下載了 '+ str(n) +' 張照片')  
    except:
      print("{}無法讀取".format(filename))

driver.quit()  ###關閉瀏覽器並退出驅動程式

八百壯士超馬系列-制霸環化100mile超馬 - 起跑點
